In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install bitsandbytes accelerate

In [2]:
import requests
import json

API_KEY = ""

response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "http://localhost",   # có thể để đại
        "X-Title": "test-app",                # tên app bất kỳ
    },
    data=json.dumps({
        "model": "meta-llama/llama-3.3-70b-instruct:free",
        "messages": [
            {
                "role": "user",
                "content": "What is the meaning of life?"
            }
        ]
    })
)

print("Status:", response.status_code)

# Nếu lỗi thì in ra luôn
if response.status_code != 200:
    print("❌ Error:", response.text)
    exit()

result = response.json()

# In toàn bộ JSON (debug)
print("\n=== RAW RESPONSE ===")
print(json.dumps(result, indent=2))

# Lấy riêng câu trả lời của AI
if "choices" in result:
    print("\n=== AI TRẢ LỜI ===\n")
    print(result["choices"][0]["message"]["content"])


Status: 200

=== RAW RESPONSE ===
{
  "id": "gen-1770019530-FGFX8LWilzsNPIZ1ShnZ",
  "provider": "OpenInference",
  "model": "meta-llama/llama-3.3-70b-instruct:free",
  "object": "chat.completion",
  "created": 1770019530,
  "choices": [
    {
      "logprobs": null,
      "finish_reason": "stop",
      "native_finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The question of the meaning of life is a complex and subjective one that has been debated by philosophers, theologians, scientists, and many others for centuries. There is no one definitive answer, as the meaning of life can vary greatly from person to person, culture to culture, and even moment to moment.\n\nSome possible perspectives on the meaning of life include:\n\n1. **Happiness and fulfillment**: For some, the meaning of life is to find happiness and fulfillment through personal relationships, achievements, and experiences.\n2. **Purpose and direction**: Others be

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-14B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


2026-02-02 07:35:08.401383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770017708.422692     454 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770017708.427926     454 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770017708.441281     454 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770017708.441296     454 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770017708.441299     454 computation_placer.cc:177] computation placer alr

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

thinking content: <think>
Okay, the user is asking for a short introduction to large language models. Let me start by defining what a large language model is. I should mention that they're AI systems trained on vast amounts of text data.

I need to explain their capabilities, like understanding and generating human-like text. Maybe include examples of tasks they can handle—text generation, translation, summarization, answering questions.

Also, it's important to touch on their training process. They use deep learning techniques, especially transformer architectures. Mention pre-training on diverse data and fine-tuning for specific tasks.

I should highlight their applications in various fields: customer service, content creation, research, etc. But also note the challenges, like computational resources and potential biases.

Make sure the introduction is concise but covers the key points. Avoid too much technical jargon since the user might not be an expert. Keep it clear and straightf

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. Configuration
model_name = "Qwen/Qwen3-14B"

# Use 4-bit quantization to fit the 14B model into the T4's VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load Tokenizer and Model
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Loading model (this may take a few minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto", # Automatically balances the model across both T4 GPUs
    trust_remote_code=True
)

# 3. Prepare the Input
prompt = "Give me a short introduction to large language models."
messages = [
    {"role": "user", "content": prompt}
]

# apply_chat_template handles the specific formatting Qwen3 needs
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True 
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# 4. Generate Response
print("\n--- Generating Response ---\n")
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048, # Reduced from 32k for faster testing on T4s
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Remove the input tokens from the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# 5. Parsing Thinking Content
# Qwen3 uses special tokens to wrap the "thought" process
THINK_END_TOKEN_ID = 151668 # ID for </think>

try:
    # Find the last occurrence of the end-thought token
    index = len(output_ids) - output_ids[::-1].index(THINK_END_TOKEN_ID)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
final_content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()

print("--- THINKING PROCESS ---")
print(thinking_content if thinking_content else "No thinking content generated.")
print("\n--- FINAL ANSWER ---")
print(final_content)

In [ ]:
import os
import subprocess
import sys
import shutil

# ==========================================
# 1. CÀI ĐẶT MÔI TRƯỜNG
# ==========================================
try:
    import llama_cpp
    print("✅ Thư viện đã có sẵn. Bỏ qua bước cài đặt.")
except ImportError:
    print("⏳ Đang cài đặt Llama.cpp và các thư viện cần thiết...")
    install_cmd = [
        sys.executable, "-m", "pip", "install", 
        "llama-cpp-python",
        "--extra-index-url", "https://abetlen.github.io/llama-cpp-python/whl/cu121",
        "--force-reinstall",
        "--no-cache-dir"
    ]
    subprocess.check_call(install_cmd)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "huggingface_hub"])

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# ==========================================
# 2. CẤU HÌNH ĐƯỜNG DẪN (Dùng /tmp để tránh giới hạn 20GB)
# ==========================================
TEMP_DIR = "/tmp/qwen_models"
CACHE_DIR = "/tmp/hf_cache"

os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

REPO_ID = "bartowski/Qwen2.5-32B-Instruct-GGUF"
FILENAME = "Qwen2.5-32B-Instruct-Q4_K_M.gguf"

def run_solution():
    print(f"\n🚀 Đang kiểm tra/tải model tại ({TEMP_DIR})...")
    
    try:
        # Tải file model
        model_path = hf_hub_download(
            repo_id=REPO_ID,
            filename=FILENAME,
            local_dir=TEMP_DIR,
            cache_dir=CACHE_DIR,
            local_dir_use_symlinks=False
        )
        print(f"✅ Model sẵn sàng tại: {model_path}")

        # Load Model vào GPU
        print("⚡ Đang nạp model vào GPU (vram)...")
        llm = Llama(
            model_path=model_path,
            n_gpu_layers=-1,      # Đẩy toàn bộ model lên GPU
            n_ctx=4096,           # Độ dài ngữ cảnh (có thể tăng nếu GPU mạnh)
            verbose=False,
            main_gpu=0            
        )

        # Câu hỏi test
        prompt = "why the sky is blue "
        
        # CHỈNH SỬA QUAN TRỌNG: System prompt ép trả lời tiếng Việt
        formatted_prompt = (
            "<|im_start|>system\n"
            "You are an AI helpful assistant"
            "<|im_end|>\n"
            f"<|im_start|>user\n{prompt}<|im_end|>\n"
            "<|im_start|>assistant\n"
        )

        print(f"\n📝 Câu hỏi: {prompt}")
        print("⏳ Đang xử lý...\n")
        
        output = llm(
            formatted_prompt,
            max_tokens=1024*4,
            temperature=0.7,
            top_p=0.9,
            stop=["<|im_end|>", "<|im_start|>"],
            echo=False
        )

        response = output['choices'][0]['text'].strip()
        
        print("="*40)
        print("🤖 KẾT QUẢ TIẾNG VIỆT:")
        print("="*40)
        print(response)
        print("="*40)

    except Exception as e:
        print(f"\n❌ LỖI: {e}")
        if "No space" in str(e):
            print("⚠️ Hết bộ nhớ /tmp. Đang dọn dẹp...")
            shutil.rmtree(TEMP_DIR, ignore_errors=True)

if __name__ == "__main__":
    run_solution()